In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from ipywidgets import VBox, ToggleButton, Button, HBox, Layout, Output, IntSlider
from IPython.display import display, clear_output

# === PARAMETERS (edit here) ===
n = 15          # Change and re-run cell to explore scaling
seed = 42
num_mc_trials = 10000  # Increase for better optimization

np.random.seed(seed)
thetas = np.random.uniform(0, 2 * np.pi, n)
vecs = np.stack((np.cos(thetas), np.sin(thetas)), axis=1)  # unit vectors

signs = np.ones(n, dtype=int)
out = Output()

# Toggle buttons (one per vector)
toggles = []
for i in range(n):
    tog = ToggleButton(value=True, description=f'{i}',
                       layout=Layout(width='38px', height='28px'),
                       tooltip=f'Vector {i} (toggle to flip sign)')
    toggles.append(tog)

def update_plot(change=None):
    global signs
    signs = np.array([1 if t.value else -1 for t in toggles])
    current_sum = signs @ vecs
    sx, sy = current_sum
    linf = max(abs(sx), abs(sy))

    with out:
        clear_output(wait=True)
        fig, ax = plt.subplots(figsize=(8, 8))
        ax.set_xlim(-6, 6)
        ax.set_ylim(-6, 6)
        ax.grid(True, alpha=0.3)
        ax.axhline(0, color='k', lw=0.8)
        ax.axvline(0, color='k', lw=0.8)
        ax.set_aspect('equal')

        # Individual signed vectors
        for i in range(n):
            col = 'blue' if signs[i] > 0 else 'red'
            v = signs[i] * vecs[i]
            ax.arrow(0, 0, v[0], v[1], head_width=0.09, color=col, alpha=0.75, lw=1.5)

        # Sum (prominent)
        ax.arrow(0, 0, sx, sy, head_width=0.25, color='lime', lw=3.5, length_includes_head=True)
        ax.text(sx + 0.3, sy + 0.3, f'SUM\n({sx:.2f}, {sy:.2f})\nℓ_∞ = {linf:.3f}',
                color='lime', fontsize=13, fontweight='bold', bbox=dict(facecolor='white', alpha=0.8))

        ax.set_title(f'Signing {n} unit vectors in ℝ² to minimize ||∑ εᵢ vᵢ||_∞\n'
                     f'Green = current sum  |  Blue/Red = signed vectors\n'
                     f'(Try toggles/buttons; boundedness visible for large n)', fontsize=14)
        plt.tight_layout()
        plt.show()

# Connect toggles
for t in toggles:
    t.observe(update_plot, names='value')

# Button callbacks
def random_signs(b):
    for t in toggles:
        t.value = bool(np.random.randint(0, 2))
    update_plot()

def greedy_optimize(b):
    curr = signs.copy()
    for _ in range(200):  # limited local search
        improved = False
        for i in range(n):
            new = curr.copy()
            new[i] *= -1
            if max(np.abs(new @ vecs)) < max(np.abs(curr @ vecs)):
                curr = new
                improved = True
        if not improved:
            break
    for j in range(n):
        toggles[j].value = bool(curr[j] > 0) # Explicitly cast to bool
    update_plot()

def monte_carlo(b):
    best_linf = float('inf')
    best_s = None
    for _ in range(num_mc_trials):
        s = np.random.choice([-1., 1.], n)
        l = max(np.abs(s @ vecs))
        if l < best_linf:
            best_linf = l
            best_s = s
    for j in range(n):
        toggles[j].value = bool(best_s[j] > 0) # Explicitly cast to bool
    update_plot()
    print(f"Best ℓ_∞ from {num_mc_trials} trials: {best_linf:.3f}")

# Buttons & slider
random_btn = Button(description='🎲 Random', button_style='info')
greedy_btn = Button(description='🔄 Greedy', button_style='success')
mc_btn = Button(description=f'🎯 MC ({num_mc_trials//1000}k)', button_style='warning')
reset_btn = Button(description='Reset (+)', button_style='primary')

random_btn.on_click(random_signs)
greedy_btn.on_click(greedy_optimize)
mc_btn.on_click(monte_carlo)
reset_btn.on_click(lambda b: [setattr(t, 'value', True) for t in toggles] or update_plot())

n_slider = IntSlider(value=n, min=5, max=50, step=1, description='n')
def on_n_change(c):
    global n, vecs, toggles
    n = int(c['new'])
    thetas = np.random.uniform(0, 2 * np.pi, n)
    vecs = np.stack((np.cos(thetas), np.sin(thetas)), axis=1)
    toggles = [ToggleButton(value=True, description=f'{i}', layout=Layout(width='38px', height='28px')) for i in range(n)]
    for t in toggles: t.observe(update_plot, names='value')
    update_plot()
n_slider.observe(on_n_change, names='value')

# Layout: controls on top, toggles in rows (~5 per row), plot below
display(HBox([n_slider, reset_btn, random_btn, greedy_btn, mc_btn]))
toggle_rows = [HBox(toggles[i:i+5]) for i in range(0, len(toggles), 5)]
for row in toggle_rows:
    display(row)
display(out)
update_plot()  # initial

Output()